In [1]:
import ee
import geemap

In [2]:
# Create map layout
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [3]:
aoi_Ttakara = ee.FeatureCollection('users/rashagh22/Tharaka_nithi')

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(aoi_Ttakara) \
    .filterDate('2015-01-01', '2019-12-31') \
    .filterMetadata('CLOUD_COVER', 'less_than', 1) \
    .median() \
    .select('B[1-7]')



vis_param = {"opacity":1,"bands":["B3","B2","B1"],"min":94,"max":2089,"gamma":1}

Map.centerObject(aoi_Ttakara, 10)
Map.addLayer(image.clip(aoi_Ttakara), vis_param, 'Landsat-8')


In [4]:
# Get the image properties
props = geemap.image_props(image)
image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -32768,
    'max': 32767},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -32768,
    'max': 32767},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -32768,
    'max': 32767},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -32768,
    'max': 32767},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': -32768,
    'max': 32767},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B6',
   'data_type': {'type': 'PixelType',
    'preci

## Creating our training dataset
- We'll sample points from our region of interest

In [5]:
training = image.sample(**{
    'region': aoi_Ttakara, # to sample only from aoi and not the whole satellite image
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(training, {}, 'training', False)
Map

Map(bottom=131495.0, center=[-0.2036244812467203, 37.87580036646076], controls=(WidgetControl(options=['positi…

In [6]:
# Instantiate the clusterer and train it.
n_clusters = 4
clusterer = ee.Clusterer.wekaCascadeKMeans(n_clusters).train(training)

In [ ]:
# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer().clip(aoi_Ttakara), {}, 'clusters')
Map

We can label the clusters

In [ ]:
legend_keys = ['One', 'Two', 'Three', 'Four']
legend_colors = ['#00FF00', '#FF0000', '#0000FF', '#FB8072']

# Reclassify the map
#result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(result.clip(aoi_Ttakara), {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters')
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')
Map

In [17]:
# Visualize the results
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

Export the results

In [18]:
import os
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'cluster.tif')

In [19]:
# You can download through url link
geemap.ee_export_image(result, filename=out_file, scale=90)

Generating URL ...
An error occurred while downloading.
Image.clipToBoundsAndScale: The geometry for image clipping must be bounded.


In [ ]:
# Or export to google drive
geemap.ee_export_image_to_drive(result, description='clusters', folder='export', scale=90)

## Reference
- https://www.youtube.com/watch?v=k9MEy2awVJQ&ab_channel=QiushengWu